# 참고자료

http://jorditorres.org/first-contact-with-tnesorflow/#cap4 (First Contact with tensorflow)

https://deepstat.tistory.com/9 (단일신경망 Single Layer Neural Network (ver. Tensorflow for Python)

https://deepstat.tistory.com/10 (단일신경망 Single Layer Neural Network (ver. Tensorflow for R)

http://fluxml.ai/ (flux: The Elegant Machine Learning Stack)

# 단일신경망 Single Layer Neural Network (ver. Flux in Julia)

## The MNIST data-set

In [1]:
using PyCall
@pyimport tensorflow.keras.datasets.mnist as MNIST

In [2]:
mnist_train, mnist_test = MNIST.load_data()
println(typeof(mnist_train))
println(typeof(mnist_test))

Tuple{Array{UInt8,3},Array{UInt8,1}}
Tuple{Array{UInt8,3},Array{UInt8,1}}


In [3]:
mnist_train[1][1,:,:]

28×28 Array{UInt8,2}:
 0x00  0x00  0x00  0x00  0x00  0x00  …  0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00  …  0xf7  0x7f  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0xc3  0x40  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x27  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00  …  0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
 0x00  0x00  0x00  0x00  0x00  0x00     0x00  0x00  0x00  0x00  0x00  0x00
   

In [4]:
mnist_test[1][1]

0x00

### `next_batch`라는 함수 만들기.

In [5]:
mutable struct My_data
    data::Tuple
    start_n::Int
end

In [6]:
function next_batch(data::My_data, n::Int)
    start_n = data.start_n
    end_n = data.start_n + n - 1
    batch_X = data.data[1][start_n:end_n,:,:]
    batch_Y = data.data[2][start_n:end_n]
    data.start_n = (end_n+1) % (size(data.data[1])[1])
    return float(reshape(batch_X,:,784)), float(batch_Y)
end

next_batch (generic function with 1 method)

In [7]:
train_dat = My_data(mnist_train,1)
test_dat = My_data(mnist_test,1);

## Single layer neural network

In [8]:
using Flux

#### 변수 지정

In [9]:
W = param(rand(10,784)./20 .- 1/40)
b = param(rand(10)./20 .- 1/40);

#### 모형 설정 Activation function : softmax

In [10]:
function model(X)
    tX = transpose(X)
    yhat = softmax(W * tX .+ b)
    return yhat
end

model (generic function with 1 method)

#### Loss function : cross-entropy

In [11]:
function cross_entropy(X,Y)
    y_onehot = Flux.onehotbatch(Y,sort(unique(Y)))
    yhat = model(X)
    loss = sum(-y_onehot .* log.(yhat))./length(Y)
    return loss
end

cross_entropy (generic function with 1 method)

#### Optimizer : ADAM (learning_rate = 0.0001)

In [12]:
PARS = params(W,b)

for i = 0:1000
    train_X, train_Y = next_batch(train_dat,6000)
    Flux.train!(cross_entropy, [(train_X, train_Y)], ADAM((PARS), 0.0001))
    if i % 100 == 0
        train_loss = cross_entropy(train_X,train_Y)
        
        test_X, test_Y = next_batch(test_dat,10000)
        tmp_test = findmax(model(test_X).data, dims=1)
        test_acc = sum([tmp_test[2][j][1] for j in 1:10000] .== (test_Y .+ 1))/10000
        
        println("step:",i,"  train_loss:" ,train_loss,"  test_acc:" ,test_acc)
    end
end

step:0  train_loss:45.543004320972074 (tracked)  test_acc:0.0763
step:100  train_loss:2.3352842586852973 (tracked)  test_acc:0.8282
step:200  train_loss:1.4340617268439977 (tracked)  test_acc:0.8699
step:300  train_loss:1.1800125983237264 (tracked)  test_acc:0.8748
step:400  train_loss:1.0417229208694117 (tracked)  test_acc:0.8797
step:500  train_loss:0.9979665354074723 (tracked)  test_acc:0.8807
step:600  train_loss:0.9151358522120402 (tracked)  test_acc:0.8831
step:700  train_loss:0.8710594189458134 (tracked)  test_acc:0.8849
step:800  train_loss:0.8412712660027076 (tracked)  test_acc:0.8837
step:900  train_loss:0.8236128599447577 (tracked)  test_acc:0.8814
step:1000  train_loss:0.8132467373551305 (tracked)  test_acc:0.8824
